In [2]:
import torch
import numpy as np

# display CUDA devices available
torch.cuda.get_device_name(0)

# set device to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
# load data

boards = np.load('../../data/seven/boards.npy')
moves = np.load('../../data/seven/moves.npy')

# shuffle data
np.random.seed(0)
np.random.shuffle(boards)
np.random.seed(0)
np.random.shuffle(moves)

# split data into train and test
TRAIN_SPLIT = 0.8
train_boards = boards[:int(len(boards) * TRAIN_SPLIT)]
train_moves = moves[:int(len(moves) * TRAIN_SPLIT)]

test_boards = boards[int(len(boards) * TRAIN_SPLIT):]
test_moves = moves[int(len(moves) * TRAIN_SPLIT):]

# convert to tensors
train_boards = torch.from_numpy(train_boards).float()
train_moves = torch.from_numpy(train_moves).float()

test_boards = torch.from_numpy(test_boards).float()
test_moves = torch.from_numpy(test_moves).float()


In [ ]:
# create model
import torch


class FeedForward(torch.nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(12 * 8 * 8 + 2, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, 256)  # 2 for turn
        self.fc4 = torch.nn.Linear(256, 128)  # 2 for turn
        self.fc5 = torch.nn.Linear(128, 64)
        self.fc6 = torch.nn.Linear(64, 1)

        self.activation = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.25)

    def forward(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc6(x)

        return x

    def predict(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.fc6(x)

        return x

    def embed(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)

        return x


# train model
loss = torch.nn.L1Loss()
model = FeedForward().to(device)


: 

In [1]:
# model graphing code
test_loss_data = []
train_loss_data = []


# hyperparameters
EPOCHS = 7000
BATCH_SIZE = 800
LEARNING_RATE = 15e-7

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# train
for epoch in range(EPOCHS):
  for i in range(0, len(train_evals), BATCH_SIZE):
    # get batch
    batch_evals = train_evals[i:i+BATCH_SIZE].to(device)
    batch_turns = train_turns[i:i+BATCH_SIZE].to(device)
    batch_boards = train_boards[i:i+BATCH_SIZE].to(device)

    # forward pass
    y_pred = model.forward(batch_boards)

    # calculate loss
    loss_val = loss(y_pred, batch_evals)

    # backward pass
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

  # print loss every 100 epochs
  if epoch % 100 == 0:
    # use test data to calculate loss
    y_pred = model.predict(test_boards.to(device))
    test_loss = loss(y_pred, test_evals.to(device)).item()*EVAL_DIVISOR

    # use train data to calculate loss
    y_pred = model.predict(train_boards.to(device))
    train_loss = loss(y_pred, train_evals.to(device)).item()*EVAL_DIVISOR
    print('Epoch: ', epoch, "test: ", test_loss, "train: ", train_loss)

    # save loss data
    test_loss_data.append(test_loss)
    train_loss_data.append(train_loss)


import matplotlib.pyplot as plt
# plot loss
plt.plot(test_loss_data, label='test')
plt.plot(train_loss_data, label='train')
plt.legend()
plt.show()



NameError: name 'torch' is not defined

In [100]:
# save model
torch.save(model.state_dict(), 'model.pth')

In [10]:
# load model
model = FeedForward().to(device)
model.load_state_dict(torch.load('model.pth'))

# test model
y_pred = model.predict(test_boards.to(device))
test_loss = loss(y_pred, test_evals.to(device)).item()*EVAL_DIVISOR
print('Test loss: ', test_loss)

Test loss:  90.45369923114777
